In [23]:
from src.molecular_structure.VibronicStates import VibronicState, OffsetOperator
from src.molecular_structure.RotationalStates import STM_RotationalBasis
from src.molecular_structure.molecular_Hamiltonians.SpinRotation import Spin_Rotation_Hamiltonian_evCaseB
from src.quantum_mechanics.AngularMomentum import ElectronicSpinBasis
from src.quantum_mechanics.Basis import OrthogonalBasis
from src.molecular_structure.molecular_Hamiltonians.Rotational_Hamiltonian import *

np.set_printoptions(formatter={'all': lambda x: f"{x.real:.2f}"},threshold=np.inf)


In [24]:
from src.group_theory.Group import *


def wavenumber_to_Hz(k):
    if isinstance(k, list):
        return [wavenumber_to_Hz(item) for item in k]
    if isinstance(k, dict):
        return {key: wavenumber_to_Hz(k[key]) for key in k}
    return k * 299792458 * 100

def wavenumber_to_GHz(k):
    if isinstance(k, list):
        return [wavenumber_to_GHz(item) for item in k]
    if isinstance(k, dict):
        return {key: wavenumber_to_GHz(k[key]) for key in k}
    return k * 299792458 * 100 / 1e9

group = C2vGroup()
A1 = C2v_A1_representation(group)
A2 = C2v_A2_representation(group)
B1 = C2v_B1_representation(group)
B2 = C2v_B2_representation(group)

In [25]:
from src.tools.SphericalTensors import SphericalTensor_prolate

X = VibronicState("X", symmetry_group=group, irrep = A1)
A = VibronicState("A", symmetry_group=group, irrep= B2)
B = VibronicState("B", symmetry_group=group, irrep= B1)
vibronic_basis = OrthogonalBasis([X,A],"Electronic")
vibronic_d_a = Operator(vibronic_basis,np.array([[0,0],[0,0]]),symmetry_group=group, irrep= A1)
vibronic_d_b = Operator(vibronic_basis,np.array([[0,1],[1,0]]),symmetry_group=group, irrep= B2)
vibronic_d_c = Operator(vibronic_basis,np.array([[0,0],[0,0]]),symmetry_group=group, irrep= B1)

# the transition dipole moment in molecule frame as a spherical tensor
vibronic_d = SphericalTensor_prolate(np.array([vibronic_d_a,vibronic_d_b,vibronic_d_c]),is_operator=True, operator_basis=vibronic_basis)

rot_basis = STM_RotationalBasis(R_range=(0,2))
es_basis = ElectronicSpinBasis(S_range=(1/2,1/2))
evr_basis = rot_basis * vibronic_basis

basis = evr_basis.couple(es_basis)
basis.rename_symbols("J","m")

In [26]:
from src.molecular_structure.RotationOperators import MShiftOperator, JShiftOperator

A_dict = {}
BC_avg2_dict = {}
BC_diff4_dict = {}

e_aa_dict = {}
e_bb_dict = {}
e_cc_dict = {}

offset_dict = {}

# X state constants
A_dict["X"] = wavenumber_to_GHz(13.05744)
BC_avg2_dict["X"] = wavenumber_to_GHz(0.296652)
BC_diff4_dict["X"] = wavenumber_to_GHz(1.8894e-3)
e_aa_dict["X"] = 45.7e-3
e_bb_dict["X"] = 32.063e-3
e_cc_dict["X"] = 41.110e-3
offset_dict["X"] = 0.0


# A state constants
A_dict["A"] = wavenumber_to_GHz(11.44854)
BC_avg2_dict["A"] = wavenumber_to_GHz(0.303107)
BC_diff4_dict["A"] = wavenumber_to_GHz(1.958e-3)
e_aa_dict["A"] = wavenumber_to_GHz(8.2369)
e_bb_dict["A"] = wavenumber_to_GHz(3.0534e-2  - 1.2617e-2 * 2)
e_cc_dict["A"] = wavenumber_to_GHz(3.0534e-2  + 1.2617e-2 * 2)
offset_dict["A"] = (wavenumber_to_Hz(15464.36739)-149e6)/1e9

# B state constants
A_dict["B"] = wavenumber_to_GHz(14.3664)
BC_avg2_dict["B"] = wavenumber_to_GHz(0.301693)
BC_diff4_dict["B"] = wavenumber_to_GHz(4.68e-3)
e_aa_dict["B"] = wavenumber_to_GHz(-7.5472)
e_bb_dict["B"] = wavenumber_to_GHz(2.083e-2 - 8.66e-3 * 2)
e_cc_dict["B"] = wavenumber_to_GHz(2.083e-2 + 8.66e-3 * 2)
offset_dict["B"] = (wavenumber_to_Hz(15885.28188)-129e6)/1e9

Z = MShiftOperator(basis) * 1e-6 + JShiftOperator(basis) * 1e-5

H_rot = Rotational_Hamiltonian_evr(basis,A_dict,BC_avg2_dict,BC_diff4_dict)

H_offset = OffsetOperator(basis,offset_dict)

H_SR = Spin_Rotation_Hamiltonian_evCaseB(basis, e_aa_dict, e_bb_dict, e_cc_dict)

H = H_rot + H_SR + H_offset + Z

In [27]:
Es, states = H.diagonalize()

In [28]:
X_states_m12 = []
X_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "X":
        X_states_m12.append(states[i])
        X_Es_m12.append(Es[i])

In [29]:
X_states_m12

[|φ_1> = 1.00 |elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5> ,
 |φ_3> = 1.00 |elec=X, S=0.5, N=1, k=0, J=0.5, m=0.5> ,
 |φ_6> = 1.00 |elec=X, S=0.5, N=1, k=0, J=1.5, m=0.5> + -3.5e-05 |elec=X, S=0.5, N=1, k=0, J=1.5, m=1.5> + 1.7e-05 |elec=X, S=0.5, N=1, k=0, J=1.5, m=-0.5> + -1.8e-06 |elec=X, S=0.5, N=2, k=-2, J=1.5, m=0.5> + 1.8e-06 |elec=X, S=0.5, N=2, k=2, J=1.5, m=0.5> ,
 |φ_10> = 1.00 |elec=X, S=0.5, N=2, k=0, J=1.5, m=0.5> + -1.8e-04 |elec=X, S=0.5, N=2, k=2, J=1.5, m=0.5> + -1.8e-04 |elec=X, S=0.5, N=2, k=-2, J=1.5, m=0.5> + 1.6e-05 |elec=X, S=0.5, N=2, k=0, J=1.5, m=1.5> + -2.1e-06 |elec=X, S=0.5, N=2, k=0, J=1.5, m=-1.5> ,
 |φ_15> = 1.00 |elec=X, S=0.5, N=2, k=0, J=2.5, m=0.5> + -1.8e-04 |elec=X, S=0.5, N=2, k=2, J=2.5, m=0.5> + -1.8e-04 |elec=X, S=0.5, N=2, k=-2, J=2.5, m=0.5> + -1.8e-06 |elec=X, S=0.5, N=2, k=0, J=2.5, m=-2.5> ,
 |φ_19> = 0.71 |elec=X, S=0.5, N=1, k=1, J=0.5, m=0.5> + -0.71 |elec=X, S=0.5, N=1, k=-1, J=0.5, m=0.5> + 1.5e-05 |elec=X, S=0.5, N=1, k=-1, J=0.5, m=-0.5

In [30]:
X_Es_m12[12]-X_Es_m12[11]

np.float64(0.08961905681246662)

In [31]:
X_Es_m12[6] - X_Es_m12[5]

np.float64(0.0651173901474067)

In [32]:
X_states_m12[6]

|φ_22> = 0.71 |elec=X, S=0.5, N=1, k=1, J=1.5, m=0.5> + -0.71 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=2, k=1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=2, k=-1, J=1.5, m=0.5> + -1.9e-05 |elec=X, S=0.5, N=1, k=1, J=1.5, m=1.5> + 1.9e-05 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=1.5> + 3.7e-06 |elec=X, S=0.5, N=1, k=1, J=1.5, m=-0.5> + -3.7e-06 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=-0.5> + -2.2e-06 |elec=X, S=0.5, N=1, k=1, J=1.5, m=-1.5> + 2.2e-06 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=-1.5> 

In [33]:
STM_basis = basis.get_uncoupled_basis()
matrix = basis.get_basis_change_matrix()
s = X_states_m12[5].change_basis(STM_basis,matrix)
s

|φ_19> = 0.58 |elec=X, S=0.5, ms=-0.5, N=1, k=1, mN=1> + -0.58 |elec=X, S=0.5, ms=-0.5, N=1, k=-1, mN=1> + -0.41 |elec=X, S=0.5, ms=0.5, N=1, k=1, mN=0> + 0.41 |elec=X, S=0.5, ms=0.5, N=1, k=-1, mN=0> + -1.2e-05 |elec=X, S=0.5, ms=0.5, N=1, k=-1, mN=-1> + 1.2e-05 |elec=X, S=0.5, ms=0.5, N=1, k=1, mN=-1> + 8.4e-06 |elec=X, S=0.5, ms=-0.5, N=1, k=-1, mN=0> + -8.4e-06 |elec=X, S=0.5, ms=-0.5, N=1, k=1, mN=0> 

In [34]:
A_states_m12 = []
A_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "X":
        X_states_m12.append(states[i])
        X_Es_m12.append(Es[i])

In [35]:
from src.molecular_structure.TDMs import DipoleOperator_evr, DipoleOperator_spin

d_evr = DipoleOperator_evr(evr_basis,vibronic_d, 0)
d_es = DipoleOperator_spin(es_basis,1)
d_uncoupled = d_evr.tensor(d_es)
d = d_uncoupled.change_basis(basis, basis.get_basis_change_matrix())

In [36]:
d.get_connected_states()

[(|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=0.5, m=0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=1.5, m=0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=A, S=0.5, N=1, k=1, J=0.5, m=0.5>),
 (|elec=X, S=0.5, N=0, k=0, J=0.5, m=0.5>,
  |elec=A, S=0.5, N=1, k=1, J=1.5, m=0.5>),
 (|elec=X, S=0.5, N=1, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=0.5, m=-0.5>),
 (|elec=X, S=0.5, N=1, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=1, k=-1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=1, k=0, J=0.5, m=-0.5>,
  |elec=A, S=0.5, N=2, k=-1, J=1.5, m=-0.5>),
 (|elec=X, S=0.5, N=1,